In [1]:
import json
import gurobipy as gp
from gurobipy import GRB

class MultiDimensionalArrayEncoder(json.JSONEncoder):
    def encode(self, obj):
        def hint_tuples(item):
            if isinstance(item, tuple):
                return {'__tuple__': True, 'items': item}
            if isinstance(item, list):
                return [hint_tuples(e) for e in item]
            if isinstance(item, dict):
                return {key: hint_tuples(value) for key, value in item.items()}
            else:
                return item

        return super(MultiDimensionalArrayEncoder, self).encode(hint_tuples(obj))
def hinted_tuple_hook(obj):
    if '__tuple__' in obj:
        return tuple(obj['items'])
    else:
        return obj
enc = MultiDimensionalArrayEncoder()

with open('parametros_preasignacion.json', encoding='utf8') as f:
    data = json.load(f, object_hook=hinted_tuple_hook)

ramos_por_curso = data['ramos_por_curso']

for curso in ramos_por_curso.keys():
    print(ramos_por_curso[curso])

# Se necesita saber a qué cursos PUEDE (nada de querer todavía) hacer clases cada profesor, dada las asignaciones previas que ya tiene. 

{'Lenguaje': 4, 'Matemáticas': 4, 'Historia': 3, 'Ciencias_Sociales': 2, 'Artes_Visuales': 2, 'Educación_Física': 2, 'Consejo_de_Curso': 1, 'Inglés': 4, 'Geografía': 3, 'Biología': 3, 'G2': 4, 'Alemán': 5, 'Física': 3, 'G3': 2}
{'Lenguaje': 4, 'Matemáticas': 4, 'Historia': 3, 'Ciencias_Sociales': 2, 'Artes_Visuales': 2, 'Educación_Física': 2, 'Consejo_de_Curso': 1, 'Inglés': 4, 'Geografía': 3, 'Biología': 3, 'G2': 4, 'Alemán': 5, 'Física': 3, 'G3': 2}


In [87]:
model = gp.Model("Preasignación de cursos a profesores")

cursos = ['4A', '4B', '5A', '5B', '8A', '8B', 'IVA', 'IVB']
profesores = ['Messi', 'Mbappé', 'Ronaldo', 'Modric', 'Perking', 'Hakimi']
ramos = ['Matemáticas', 'Lenguaje', 'Historia', 'Alemán']

# ! Otros conjuntos importantes
clases_ya_asignadas = {
    ('Matemáticas','Messi', '4A')
}
cantidad_de_horas_por_ramo_por_curso = {
    '4A': {
        "Orientación": 1,
        "Educación_Física": 3,
        "Lenguaje": 7,
        "Matemáticas": 6,
        "Alemán": 6,
        "Ciencias_Naturales": 3,
        "Religión": 2,
        "Historia": 3,
        "Taller_Comprensión_Lectora": 1,
        "Música": 2,
        "Tecnología": 2,
        "Artes_Visuales": 2
    },
    '4B': {
        "Orientación": 1,
        "Educación_Física": 3,
        "Lenguaje": 7,
        "Matemáticas": 6,
        "Alemán": 6,
        "Ciencias_Naturales": 3,
        "Religión": 2,
        "Historia": 3,
        "Taller_Comprensión_Lectora": 1,
        "Música": 2,
        "Tecnología": 2,
        "Artes_Visuales": 2
    },
    '5A': {
        "Orientación": 1,   
        "Lenguaje": 6,
        "Inglés": 4,
        "Matemáticas": 6,
        "Alemán": 6,
        "Ciencias_Naturales": 3,
        "Geografía": 2,
        "Historia": 2,
        "Educación_Física": 2,
        "Religión": 2,
        "Música": 2,
        "Artes_Visuales": 2,
        "Tecnología": 2
    },
    '5B': {
        "Orientación": 1,   
        "Lenguaje": 6,
        "Inglés": 4,
        "Matemáticas": 6,
        "Alemán": 6,
        "Ciencias_Naturales": 3,
        "Geografía": 2,
        "Historia": 2,
        "Educación_Física": 2,
        "Religión": 2,
        "Música": 2,
        "Artes_Visuales": 2,
        "Tecnología": 2
    },
    '8A': {
        "Matemáticas": 6,
        "Orientación": 1,
        "Religión": 2,
        "Alemán": 6,
        "Lenguaje": 6,
        "Tecnología": 2,
        "Educación_Física": 2,
        "Química": 2,
        "Historia": 4,
        "Biología": 2,
        "Física": 2, 
        "Inglés": 4,
        "Música": 2
    },
    '8B': {
        "Matemáticas": 6,
        "Orientación": 1,
        "Religión": 2,
        "Alemán": 6,
        "Lenguaje": 6,
        "Tecnología": 2,
        "Educación_Física": 2,
        "Química": 2,
        "Historia": 4,
        "Biología": 2,
        "Física": 2, 
        "Inglés": 4,
        "Música": 2
    },
    'IVA': {
        "Lenguaje": 4,
        "Matemáticas": 4,
        "Historia": 3,
        "Ciencias_Sociales": 2,
        "Artes_Visuales": 2,
        "Educación_Física": 2,
        "Consejo_de_Curso": 1,
        "Inglés": 4,
        "Geografía": 3,
        "Biología": 3,
        "G2": 4,
        "Alemán": 5,
        "Física": 3,
        "G3": 2
    },
    'IVB': {
        "Lenguaje": 4,
        "Matemáticas": 4,
        "Historia": 3,
        "Ciencias_Sociales": 2,
        "Artes_Visuales": 2,
        "Educación_Física": 2,
        "Consejo_de_Curso": 1,
        "Inglés": 4,
        "Geografía": 3,
        "Biología": 3,
        "G2": 4,
        "Alemán": 5,
        "Física": 3,
        "G3": 2
    },
}
disponibilidad_horaria ={
    "Messi": 25,
    "Mbappé": 25,
    "Ronaldo": 25,
    "Modric": 25,
    "Perking": 20,
    "Hakimi": 25
}
ramos_y_cursos_dictados_por_profesor = {
    'Messi': [
        ('Matemáticas', '4A'), 
        ('Matemáticas', '5A'),
        ('Matemáticas', '8A'),
        ('Matemáticas', 'IVA')
    ],
    'Mbappé': [
        ('Lenguaje', '4A'),
        ('Lenguaje', '4B'),
        ('Lenguaje', '5A'),
        ('Lenguaje', '5B'),
        ('Lenguaje', '8A'),
        ('Lenguaje', '8B'),
        ('Lenguaje', 'IVA'),
        ('Lenguaje', 'IVB'),
    ],
    'Ronaldo': [
        ('Historia', '4A'),
        ('Historia', '4B'),
        ('Historia', '5A'),
        ('Historia', '5B'),
        ('Historia', '8A'),
        ('Historia', '8B'),
        ('Historia', 'IVA'),
        ('Historia', 'IVB'),
    ],
    'Modric': [
        ('Alemán', '4A'),
        ('Alemán', '4B'),
        ('Alemán', '5A'),
        ('Alemán', '5B'),
        ('Alemán', '8A'),
        ('Alemán', '8B'),
        ('Alemán', 'IVA'),
        ('Alemán', 'IVB'),
    ],
    'Perking': [
        ('Alemán', '4A'),
        ('Alemán', '4B'),
        ('Alemán', '5A'),
        ('Alemán', '5B'),
        ('Alemán', '8A'),
        ('Alemán', '8B'),
        ('Alemán', 'IVA'),
        ('Alemán', 'IVB'),
    ],
    'Hakimi': [
        ('Matemáticas', '4B'), 
        ('Matemáticas', '5B'),
        ('Matemáticas', '8B'),
        ('Matemáticas', 'IVB')
    ]
}

y = model.addVars(profesores, ramos, cursos, vtype=GRB.BINARY, name="y")
w = model.addVars(ramos, cursos, vtype=GRB.BINARY, name = "w")

In [88]:
model.addConstrs((sum(y[p,r,c] for p in profesores) + w[r,c] == 1 for r in ramos for c in cursos),name="R1");

In [89]:
model.addConstrs((sum(y[p,r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for r in ramos for c in cursos) <= disponibilidad_horaria[p] for p in profesores),name="R2");

In [90]:
model.addConstrs((y[p,r,c] == 0 for p in profesores for r,c in list(set([(a,b) for a in ramos for b in cursos]) - set(ramos_y_cursos_dictados_por_profesor[p]))),name="R4");

In [91]:
model.addConstrs((y[p,r,c] == 1 for r,p,c in clases_ya_asignadas),name="R5");

In [92]:
obj = 50 * sum(w[r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for r in ramos for c in cursos) # ? Minimizar la cantidad de cursos asignados que hay que parchar
model.setObjective(obj, GRB.MINIMIZE)



model.update()

# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.optimize()
model.write('out_preasignacion.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 191 rows, 224 columns and 569 nonzeros
Model fingerprint: 0x1e7e30f0
Variable types: 0 continuous, 224 integer (224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+02, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 7700.0000000
Presolve removed 181 rows and 208 columns
Presolve time: 0.00s
Presolved: 10 rows, 16 columns, 32 nonzeros
Found heuristic solution: objective 1300.0000000
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1300.00000 1300.00000  0.00%     -    0s

Explor

In [113]:
with open ('out_preasignacion.sol') as f:
    profesores_faltantes = []
    ocupaciones_profesores = []
    for line in f.readlines()[2:]:
        if line[-2] == '1' and line[0] == 'w':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            profesores_faltantes.append(line)
        if line[-2] == '1' and line[0] == 'y':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            ocupaciones_profesores.append(line)

suma = 0
for elem in profesores_faltantes:
    suma += cantidad_de_horas_por_ramo_por_curso[elem[1]][elem[0]]
    print(f'Al curso {elem[1]} no fue posible asignarle profesor de {elem[0]} ({cantidad_de_horas_por_ramo_por_curso[elem[1]][elem[0]]} horas)')
print(f'El total de horas que no se pudieron asignar fue de {suma} horas')
horas_placeholder = 1.5 * suma # ! Esta es una estimación que considera la cantidad de horas placeholder 
                               # ! que faltarían y a la vez deja espacio para las prioridades


suma = 0
for i in range(len(ocupaciones_profesores)):
    if i+1 != len(ocupaciones_profesores):
        if ocupaciones_profesores[i][0] == ocupaciones_profesores[i+1][0]:
            suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
        if ocupaciones_profesores[i][0] != ocupaciones_profesores[i+1][0]:
            suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
            print(f'Profesor {ocupaciones_profesores[i][0]} tiene {suma} horas asignadas')
            suma = 0
    if i+1 == len(ocupaciones_profesores):
        suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
        print(f'Profesor {ocupaciones_profesores[i][0]} tiene {suma} horas asignadas')


Al curso 4B no fue posible asignarle profesor de Lenguaje (7 horas)
Al curso 5B no fue posible asignarle profesor de Lenguaje (6 horas)
Al curso IVA no fue posible asignarle profesor de Lenguaje (4 horas)
Al curso IVB no fue posible asignarle profesor de Lenguaje (4 horas)
Al curso IVB no fue posible asignarle profesor de Alemán (5 horas)
El total de horas que no se pudieron asignar fue de 26 horas
Profesor Messi tiene 22 horas asignadas
Profesor Mbappé tiene 25 horas asignadas
Profesor Ronaldo tiene 24 horas asignadas
Profesor Modric tiene 24 horas asignadas
Profesor Perking tiene 17 horas asignadas
Profesor Hakimi tiene 22 horas asignadas


In [117]:
model2 = gp.Model("Asignación de cursos a profesores")

preferencias_profesores = {
    "Messi": [
            ('Matemáticas', '8A'),
            ('Matemáticas', 'IVA')
    ],
    'Mbappé': [
        ('Lenguaje', 'IVA'),
        ('Lenguaje', 'IVB')
    ],
    'Ronaldo': [
        ('Historia', '8A'),
        ('Historia', '8B')
    ],
    'Modric': [
        ('Alemán', '4A'),
        ('Alemán', '4B')
    ],
    'Perking': [
        ('Alemán', '4B'),
        ('Alemán', '5A')
    ],
    'Hakimi': [
        ('Matemáticas', '4B'), 
        ('Matemáticas', '5B')
    ]
}

profesores_placeholder = profesores.copy()
profesores_placeholder.append('Placeholder')

z = model2.addVars(profesores_placeholder, ramos, cursos, vtype=GRB.BINARY, name="z")

In [118]:
model2.addConstrs((sum(z[p,r,c] for p in profesores_placeholder) == 1 for r in ramos for c in cursos),name="R1");

In [119]:
model2.addConstrs((sum(z[p,r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for r in ramos for c in cursos) <= disponibilidad_horaria[p] for p in profesores),name="R2");

In [120]:
model2.addConstr((sum(z['Placeholder',r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for r in ramos for c in cursos) <= horas_placeholder),name="R3");

In [121]:
model2.addConstrs((z[p,r,c] == 0 for p in profesores for r,c in list(set([(a,b) for a in ramos for b in cursos]) - set(ramos_y_cursos_dictados_por_profesor[p]))),name="R4");

In [122]:
model2.addConstrs((z[p,r,c] == 1 for r,p,c in clases_ya_asignadas),name="R5");

In [124]:
preferencias_profesores

{'Messi': [('Matemáticas', '8A'), ('Matemáticas', 'IVA')],
 'Mbappé': [('Lenguaje', 'IVA'), ('Lenguaje', 'IVB')],
 'Ronaldo': [('Historia', '8A'), ('Historia', '8B')],
 'Modric': [('Alemán', '4A'), ('Alemán', '4B')],
 'Perking': [('Alemán', '4B'), ('Alemán', '5A')],
 'Hakimi': [('Matemáticas', '4B'), ('Matemáticas', '5B')]}

In [126]:
obj2 = sum(z[p,r,c] for p in profesores for r,c in preferencias_profesores[p])
model2.setObjective(obj2, GRB.MAXIMIZE)

model2.update()
model2.optimize()
model2.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 192 rows, 224 columns and 601 nonzeros
Model fingerprint: 0xbeb8587b
Variable types: 0 continuous, 224 integer (224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]

Loaded MIP start from previous solve with objective 11

Presolve removed 183 rows and 205 columns
Presolve time: 0.00s
Presolved: 9 rows, 19 columns, 38 nonzeros
Variable types: 0 continuous, 19 integer (16 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 11 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.100000000000e+01, best bound 1.100000000000e+01, gap 0.0000%


In [139]:
with open ('out.sol') as f:
    ocupaciones_profesores = []
    for line in f.readlines()[2:]:
        if line[-2] == '1' and line[0] == 'z':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            ocupaciones_profesores.append(line)

for elem in ocupaciones_profesores:
    if elem[0] != 'Placeholder':
        if (elem[1], elem[2]) in preferencias_profesores[elem[0]]:
            print(elem)

['Messi', 'Matemáticas', '8A']
['Messi', 'Matemáticas', 'IVA']
['Mbappé', 'Lenguaje', 'IVA']
['Mbappé', 'Lenguaje', 'IVB']
['Ronaldo', 'Historia', '8A']
['Ronaldo', 'Historia', '8B']
['Modric', 'Alemán', '4A']
['Perking', 'Alemán', '4B']
['Perking', 'Alemán', '5A']
['Hakimi', 'Matemáticas', '4B']
['Hakimi', 'Matemáticas', '5B']
